In [1]:
import numpy as np 
import boto3 
import sagemaker 
import io 
import sagemaker.amazon.common as smac 
import os
import pandas as pd 


#Read the csv from s3.
#download from s3 bucker the bike share data CSV file based on the publically available bike share data
from io import StringIO 
s3 = boto3.resource('s3') 
bucket = 'sagemaker-us-east-1-895550936162'
object_key = 'day.csv'

#Load the data into a pandas dataframe
csv_obj = s3.Object(bucket, object_key) 
csv_string = csv_obj.get()['Body'].read().decode('utf-8') 

dataset = pd.read_csv(StringIO(csv_string)) 
dataset.head() 


[03/03/25 18:03:47] INFO     Skipping checksum validation.   ]8;id=537358;file:///opt/conda/lib/python3.11/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=175095;file:///opt/conda/lib/python3.11/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             Response did not contain one of                    
                             the following algorithms:                          
                             ['crc32', 'sha1', 'sha256'].                       


,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [2]:
#Linear learner does not like catgeorical fied like dteday so we need to convert the categorical data field
dataset['dteday'] = dataset['dteday'].str.replace("-","") 
dataset.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,20110101,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,20110102,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,20110103,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,20110104,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,20110105,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [3]:
#We need to now randomize the data and split it between train and test datasets on a 70/30 split
train_data, test_data = np.split(dataset.sample(frac=1, random_state=1729), [int(0.7 * len(dataset))]) 
print(train_data.shape, test_data.shape)

(511, 16) (220, 16)


/opt/conda/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [6]:
#Now we extract the features and labels and put that on a dataset
feature_dataset = train_data[['instant','dteday','season','yr','mnth','holiday','weekday','workingday',
                              'weathersit','temp','atemp','hum','windspeed','casual','registered']]
features = np.array(feature_dataset.values).astype('float32') 

label_dataset = train_data[['cnt']]
labels = np.array(label_dataset.values).astype('float32') 
labels_vec = np.squeeze(np.asarray(labels))

In [8]:
#Setup protoBuffer
buffer = io.BytesIO() 
smac.write_numpy_to_dense_tensor(buffer, features, labels_vec) 
buffer.seek(0) 

prefix = 'realestate' 
key = 'linearregression' 
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buffer) 
s3_training_data_location = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('training dataset will be uploaded to: {}'.format(s3_training_data_location)) 


training dataset will be uploaded to: s3://sagemaker-us-east-1-895550936162/realestate/train/linearregression


In [10]:
output_location = 's3://{}/{}/output'.format(bucket, prefix) 
print('model artifacts will be uploaded to: {}'.format(output_location))

model artifacts will be uploaded to: s3://sagemaker-us-east-1-895550936162/realestate/output


In [14]:
#Get the Linear Learner container instance
from sagemaker.amazon.amazon_estimator import get_image_uri 
linear_container = get_image_uri(boto3.Session().region_name, 'linear-learner') 

[03/03/25 18:21:03] WARNING  The method get_image_uri has     ]8;id=693959;file:///opt/conda/lib/python3.11/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=199209;file:///opt/conda/lib/python3.11/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             been renamed in sagemaker>=2.                      
                             See:                                               
                             https://sagemaker.readthedocs.io                   
                             /en/stable/v2.html for details.                    
                    INFO     Same images used for training and ]8;id=834356;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=460484;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#393\393]8;;\
                             inference. Defaulting to image                     
                             scope: inference.    

In [19]:
#Now we have everything setup we now need to train our model
from sagemaker import get_execution_role 

role = get_execution_role() 

sagemaker_session = sagemaker.Session()

#Provide the container, role, instance type and model output location
linear = sagemaker.estimator.Estimator(linear_container,
                                       role=role,
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge', 
                                       output_path=output_location,
                                       sagemaker_session=sagemaker_session) 
#Provide the number of features identified during the data preparation 
#Provide the predictor_type 

linear.set_hyperparameters(feature_dim=15,
                           mini_batch_size=4, 
                           predictor_type='regressor') 
#Train the model using the previosuly prepared test data and validate the data by providing the validation data.
linear.fit({'train': s3_training_data_location}) 

[03/03/25 18:52:05] WARNING  train_instance_count has been    ]8;id=103902;file:///opt/conda/lib/python3.11/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=434153;file:///opt/conda/lib/python3.11/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             renamed in sagemaker>=2.                           
                             See:                                               
                             https://sagemaker.readthedocs.io                   
                             /en/stable/v2.html for details.                    
                    WARNING  train_instance_type has been     ]8;id=388813;file:///opt/conda/lib/python3.11/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=626654;file:///opt/conda/lib/python3.11/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             renamed in sagemaker>=2.                           
                             See:              

In [20]:
#Deploy the model
linear_predictor = linear.deploy(initial_instance_count=1,
                                 instance_type='ml.c4.xlarge',
                                 endpoint_name='bikeshare-sagemaker-regression-v1')

[03/03/25 19:11:18] INFO     Creating model with name:           ]8;id=423682;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=743662;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4094\4094]8;;\
                             linear-learner-2025-03-03-19-11-18-                
                             655                                                
[03/03/25 19:11:19] INFO     Creating endpoint-config with name  ]8;id=171994;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=940412;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#5889\5889]8;;\
                             bikeshare-sagemaker-regression-v1                  
                    INFO     Creating endpoint with name         ]8;id=879320;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=381682;file:///opt/conda/lib/python3.11/site-package

In [22]:
#Get prediction using the test data
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer


linear_predictor.content_type = 'text/csv' 
linear_predictor.serializer = CSVSerializer() 
linear_predictor.deserializer = JSONDeserializer()

test_feature_dataset = test_data[['instant','dteday','season','yr','mnth','holiday','weekday','workingday',
                              'weathersit','temp','atemp','hum','windspeed','casual','registered']]

test_actuals = np.array(test_data['cnt'].astype('float32'))
test_features = np.array(test_feature_dataset.values.astype('float32')) 

predictions = [] 
actuals = []
for tf, actual in zip(test_features, test_actuals):
    prediction = linear_predictor.predict(tf) 
    predictions.append(prediction['predictions'][0]['score']) 
    actuals.append(actual) 
    print('prediction: ', prediction['predictions'][0]['score'], '\t\tactual: ', str(actual)) 
    

prediction:  799.390625 		actual:  801.0
prediction:  5216.6875 		actual:  5217.0
prediction:  7765.5625 		actual:  7767.0
prediction:  6850.9375 		actual:  6852.0
prediction:  2209.96875 		actual:  2209.0
prediction:  6289.28125 		actual:  6290.0
prediction:  4790.703125 		actual:  4792.0
prediction:  1866.796875 		actual:  1865.0
prediction:  5670.375 		actual:  5668.0
prediction:  4492.25 		actual:  4492.0
prediction:  4368.203125 		actual:  4367.0
prediction:  2402.453125 		actual:  2402.0
prediction:  3847.546875 		actual:  3846.0
prediction:  4787.96875 		actual:  4788.0
prediction:  3189.015625 		actual:  3190.0
prediction:  3006.515625 		actual:  3005.0
prediction:  7284.625 		actual:  7286.0
prediction:  2131.234375 		actual:  2132.0
prediction:  5462.21875 		actual:  5464.0
prediction:  4984.09375 		actual:  4985.0
prediction:  6304.625 		actual:  6304.0
prediction:  5532.03125 		actual:  5532.0
prediction:  8008.171875 		actual:  8009.0
prediction:  3000.46875 		actual:  299

In [23]:
#Get accuracy using Cosine Similarity method
from numpy import dot 
from numpy.linalg import norm
tolerance = 1e-10 
accuracy = (dot(actuals, predictions)/(norm(actuals)*norm(predictions)))*100 
print('accuracy: ', accuracy)

accuracy:  100.0000031367257
